# Efficient Python Codes

In this lecture we will study ways to optimize python codes

## Timing and profiling code

progress bar:

In [18]:
from time import sleep

from tqdm import tqdm, trange, tqdm_notebook

In [19]:
for i in tqdm(range(10)):
    
    sleep(.1)

100%|██████████| 10/10 [00:01<00:00,  9.75it/s]


In [20]:
def generator():
    
    for i in range(10):
        
        yield i

In [21]:
for i in tqdm(generator()):
    
    sleep(.1)

10it [00:01,  9.75it/s]


In [22]:
for i in tqdm(generator(), total=10):
    
    sleep(.1)

100%|██████████| 10/10 [00:01<00:00,  9.72it/s]


In [10]:
for i in trange(10):
    
    sleep(.1)

100%|██████████| 10/10 [00:01<00:00,  9.76it/s]


In [12]:
for i in tqdm_notebook(range(10)):
    
    sleep(.1)

/home/pythoncourse/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [13]:
import pandas as pd

In [14]:
tqdm.pandas()

/home/pythoncourse/miniconda3/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [15]:
amazon_fires = pd.read_csv('data/amazon.csv', encoding='latin1')

In [16]:
amazon_fires.head()

,Ano,Estado,Mês,Número,Período
0,1998,Acre,Janeiro,0.0,01-01-1998
1,1999,Acre,Janeiro,0.0,01-01-1999
2,2000,Acre,Janeiro,0.0,01-01-2000
3,2001,Acre,Janeiro,0.0,01-01-2001
4,2002,Acre,Janeiro,0.0,01-01-2002


In [17]:
amazon_fires.progress_apply(lambda x: '{} {}'.format(x['Estado'], x['Mês']), axis='columns')

100%|██████████| 6454/6454 [00:00<00:00, 47412.77it/s]


0             Acre Janeiro
1             Acre Janeiro
2             Acre Janeiro
3             Acre Janeiro
4             Acre Janeiro
               ...        
6449    Tocantins Dezembro
6450    Tocantins Dezembro
6451    Tocantins Dezembro
6452    Tocantins Dezembro
6453    Tocantins Dezembro
Length: 6454, dtype: object

## Timing a code

Magic commands or magic functions are one of the important enhancements that IPython offers compared to the standard Python shell. These magic commands are intended to solve common problems in data analysis using Python. In fact, they control the behaviour of IPython itself.

In [28]:
def good_practice(rand_array):
    bigger_than_fifties = [*rand_array[rand_array > 50]] # using masking, broadcasting and unpacking over an np.array
    return bigger_than_fifties

def bad_practice(rand_list):
    bigger_than_fifties = []
    for i in range(len(rand_list)):
        if rand_list[i] > 50:
            bigger_than_fifties.append(rand_list[i])
    return bigger_than_fifties

In [5]:
import numpy as np

rand_array = np.random.randint(100, size=1000)
rand_list = [*rand_array]

The Magic commands time and timeit 

In [36]:
%time a = bad_practice(rand_list)

CPU times: user 1.18 ms, sys: 45 µs, total: 1.22 ms
Wall time: 1.23 ms


In [37]:
%timeit a = bad_practice(rand_list)

206 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
%time a = good_practice(rand_array)

CPU times: user 1.44 ms, sys: 45 µs, total: 1.48 ms
Wall time: 2.17 ms


In [42]:
%timeit -n 100 a = good_practice(rand_array)

54.2 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%time

a=bad_practice(rand_list)
b=good_practice(rand_array)

CPU times: user 3.09 ms, sys: 100 µs, total: 3.19 ms
Wall time: 3.3 ms


In [44]:
%%timeit

a=bad_practice(rand_list)
b=good_practice(rand_array)

250 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
%%timeit -n 100

a=bad_practice(rand_list)
b=good_practice(rand_array)

291 µs ± 93.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
%prun bad_practice(rand_list)

In [48]:
%prun good_practice(rand_array)

In [ ]:
# conda install -c anaconda line_profiler 

In [50]:
%load_ext line_profiler

In [51]:
%lprun good_practice(rand_array)

In [52]:
%lprun -f good_practice good_practice(rand_array)

In [53]:
%lprun -f bad_practice bad_practice(rand_list)

## Memory profiling

In [ ]:
# conda install -c anaconda memory_profiler 

In [3]:
%load_ext memory_profiler

In [55]:
%memit good_practice(rand_array)

peak memory: 141.88 MiB, increment: 0.05 MiB


In [56]:
%memit bad_practice(rand_list)

peak memory: 142.08 MiB, increment: 0.00 MiB


In [57]:
%%memit

a=bad_practice(rand_list)
b=good_practice(rand_array)

peak memory: 142.08 MiB, increment: 0.00 MiB


In [58]:
%mprun -f good_practice good_practice(rand_array)

ERROR: Could not find file <ipython-input-28-0718842debb0>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.



In [59]:
%%file practices.py

def good_practice(rand_array):
    bigger_than_fifties = [*rand_array[rand_array > 50]] # using masking, broadcasting and unpacking over an np.array
    return bigger_than_fifties

def bad_practice(rand_list):
    bigger_than_fifties = []
    for i in range(len(rand_list)):
        if rand_list[i] > 50:
            bigger_than_fifties.append(rand_list[i])
    return bigger_than_fifties

Writing practices.py


In [1]:
from practices import good_practice, bad_practice

In [6]:
%mprun -f good_practice good_practice(rand_array)

In [7]:
%mprun -f bad_practice bad_practice(rand_list)

How to optimize? AVOID LOOPS AND CONDITIONS!

1. Prefer numpy arrays, pandas apply, itertools and collections.
2. Try list comprehensions.
3. Write better loops.

## Numba

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

You don't need to replace the Python interpreter, run a separate compilation step, or even have a C/C++ compiler installed. Just apply one of the Numba decorators to your Python function, and Numba does the rest. 

In [10]:
from numba import jit

import numpy as np

import random
import numpy.random as rd

In [11]:
@jit(nopython=True)
def numba_monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [12]:
def numpy_monte_carlo_pi(nsamples):
    
    x = rd.uniform(size=nsamples)
    y = rd.uniform(size=nsamples)

    acc = ((x ** 2 + y ** 2) < 1.0).sum()

    return 4.0 * acc / nsamples

In [13]:
%%time

numba_monte_carlo_pi(100000)

CPU times: user 212 ms, sys: 0 ns, total: 212 ms
Wall time: 222 ms


3.14192

In [14]:
%%time

numpy_monte_carlo_pi(100000)

CPU times: user 8.39 ms, sys: 4.06 ms, total: 12.4 ms
Wall time: 62.8 ms


3.1414

In [15]:
%%time

numba_monte_carlo_pi(100000)

CPU times: user 5.23 ms, sys: 174 µs, total: 5.41 ms
Wall time: 5.81 ms


3.1502

## High Performance Computing

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

With Numba:

In [16]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, parallel=True)
def gaussians(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result / SQRT_2PI / n

In [18]:
means = np.random.uniform(-1, 1, size=1000000)
widths = np.random.uniform(0.1, 0.3, size=1000000)

gaussians(0.4, means, widths)

array([2.04124505e-07, 1.68985279e-12, 2.26282630e-06, ...,
       4.69602473e-07, 2.18915514e-07, 4.29909344e-07])

In [19]:
gaussians_nothread = jit(nopython=True)(gaussians.py_func)

gaussians_nothread(0.4, means, widths)

array([2.04124505e-07, 1.68985279e-12, 2.26282630e-06, ...,
       4.69602473e-07, 2.18915514e-07, 4.29909344e-07])

In [20]:
%timeit gaussians.py_func(0.4, means, widths) # compare to pure NumPy
%timeit gaussians_nothread(0.4, means, widths)
%timeit gaussians(0.4, means, widths)

7.64 ms ± 24 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15 ms ± 31.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.05 ms ± 50 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
from numba import prange

In [22]:
# Serial version
@jit(nopython=True)
def monte_carlo_pi_serial(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

# Parallel version
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in prange(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [23]:
%time monte_carlo_pi_serial(int(4e8))
%time monte_carlo_pi_parallel(int(4e8))

CPU times: user 4.03 s, sys: 8.21 ms, total: 4.04 s
Wall time: 4.05 s
CPU times: user 7.51 s, sys: 7.49 ms, total: 7.52 s
Wall time: 2.09 s


3.14167166

### External Multithreading

Sometimes your threading system is external to Numba entirely. You might be using concurrent.futures to run functions in multiple threads, or a parallel framework like Dask. For these situations, you do not want to use ParallelAccelerator, but do want to allow the Numba-compiled function to run concurrently in different threads.

To do this, you want the Numba function to release the Global Interpreter Lock (GIL) during execution. This can be done using the nogil=True option to @jit.

Let's do our Monte Carlo example again, but with Dask. Note that Numba will still handle initializing separate random number generator seeds on each thread, as it did with ParallelAccelerator.

In [24]:
import dask
import dask.delayed

In [25]:
@jit(nopython=True, nogil=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

monte_carlo_pi(int(1e6))

3.139856

In [26]:
delayed_monte_carlo_pi = dask.delayed(monte_carlo_pi)

In [27]:
futures = [delayed_monte_carlo_pi(int(4e8)) for i in range(4)]

futures

[Delayed('monte_carlo_pi-5710ae24-8db8-4196-aaca-c71caff582ff'),
 Delayed('monte_carlo_pi-a368e99f-8aab-4554-8bb2-4f124d723810'),
 Delayed('monte_carlo_pi-a907c829-2fdb-42ca-b49c-014ae52a59e5'),
 Delayed('monte_carlo_pi-2fcda71c-7c84-4121-b072-ad115a7f5f58')]

In [31]:
dask.compute(futures, num_workers=1)[0]

[3.14162176, 3.14153673, 3.14161754, 3.1415316]

In [ ]:
%%time

results = dask.compute(futures)[0]

np.sum(results)/4

In [ ]:
%%time
futures = [delayed_monte_carlo_pi(int(4e8)) for i in range(4)]
results = dask.compute(futures, num_workers=1)[0]

np.sum(results)/4

### Pymp

In [32]:
import pymp

In [33]:
a = []

with pymp.Parallel(4) as p:
    
    for i in p.range(40):
        
        a.append(i**4)

In [34]:
a

[0, 1, 16, 81, 256, 625, 1296, 2401, 4096, 6561]

In [35]:
r = []

with pymp.Parallel(4) as p:
    
    for i in p.iterate(a):
        
        r.append(i**4)

In [36]:
r

[]

In [37]:
r = pymp.shared.list()

with pymp.Parallel(4) as p:
    
    for i in p.iterate(a):
        
        r.append(i**4)

In [38]:
r

<ListProxy object, typeid 'list' at 0x7fad4811f450>

In [39]:
list(r)

[0,
 1,
 65536,
 43046721,
 4294967296,
 152587890625,
 2821109907456,
 33232930569601,
 281474976710656,
 1853020188851841]